<a href="https://colab.research.google.com/github/Taka0007/2023-seminar-previous.term/blob/main/code/%E9%9F%B3%E5%A3%B0%E5%88%86%E9%9B%A2NMF%E5%AE%9F%E8%A3%85%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [2]:
# wavファイルDL
!wget https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav -O aiueo.wav
file_path = '/content/aiueo.wav'
audio_data, sr = librosa.load(file_path)

--2023-06-06 08:58:47--  https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav [following]
--2023-06-06 08:58:48--  https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32832 (32K) [audio/wav]
Saving to: ‘aiueo.wav’

aiueo.wav           100%[===================>]  32.06K  --.-KB/s    in 0.007s  

2023-06-06 08:58:48 (4.72 MB/s) - ‘aiueo.wav’ saved [32832/32832]



<ipython-input-2-0da643ce9416>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [3]:
#高速フーリエ変換
fft_spec = np.fft.rfft(audio_data)
print(fft_spec)

[-2.78108568e+01+0.00000000e+00j -4.17856923e+00+1.67983792e+01j
  2.63063313e+01+1.49804326e+01j ...  6.29508110e-03-1.11692273e-06j
  6.29458866e-03+2.31351483e-06j  6.29570565e-03-2.41347811e-07j]


In [4]:
type(fft_spec)

numpy.ndarray

In [ ]:
#非負値行列因子分解(NMF)の実行

class NMF:
  """
  非負値行列因子分解を行う
  X を H,Uの2行列に分解する
  HUがXに近づくように近似を行う

  Attributes
    --------

    __init__:None
    
    update:

    """
    def __init__(self, X : np.array)-> None:
      # 元の行列を代入
      self.X = X
      
      #行列の列数と行数を取得
      self.numRows = X.shape[0]
      self.numColumns = X.shape[1]

      #分解後の行列の初期値策定
    
    def 


    def update(self) -> np.array:




In [ ]:
NMF = NMF